In [1]:
pip install torch torchvision matplotlib efficientnet_pytorch


  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached 

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt


In [3]:
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Loading and splitting the dataset
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

total_train = len(trainset)
val_size = int(0.1 * total_train)
train_size = total_train - val_size
train_dataset, val_dataset = torch.utils.data.random_split(trainset, [train_size, val_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


100%|██████████| 169001437/169001437 [00:04<00:00, 33923761.26it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
class CIFAR100EfficientNet(nn.Module):
    def __init__(self):
        super(CIFAR100EfficientNet, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        num_ftrs = self.efficientnet._fc.in_features
        self.efficientnet._fc = nn.Linear(num_ftrs, 100)

    def forward(self, x):
        return self.efficientnet(x)

model = CIFAR100EfficientNet()


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 166MB/s]

Loaded pretrained weights for efficientnet-b0


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [6]:
def train(model, trainloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[Epoch, Batch]: [{epoch+1}, {i+1}], Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

def validate(model, valloader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100. * correct / total
    print(f'Validation Loss: {val_loss / len(valloader):.3f}, Accuracy: {accuracy:.2f}%')
    return val_loss / len(valloader), accuracy


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

early_stopping_patience = 5
early_stopping_counter = 0
min_val_loss = float('inf')

for epoch in range(10):  # Adjust the number of epochs as needed
    train(model, trainloader, criterion, optimizer, device)
    val_loss, _ = validate(model, valloader, criterion, device)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[Epoch, Batch]: [1, 100], Loss: 4.212
[Epoch, Batch]: [1, 200], Loss: 2.346
[Epoch, Batch]: [1, 300], Loss: 1.293


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation Loss: 1.040, Accuracy: 69.80%
[Epoch, Batch]: [2, 100], Loss: 0.911
[Epoch, Batch]: [2, 200], Loss: 0.813
[Epoch, Batch]: [2, 300], Loss: 0.765
Validation Loss: 0.735, Accuracy: 77.26%
[Epoch, Batch]: [3, 100], Loss: 0.614
[Epoch, Batch]: [3, 200], Loss: 0.613
[Epoch, Batch]: [3, 300], Loss: 0.570
Validation Loss: 0.641, Accuracy: 80.54%
[Epoch, Batch]: [4, 100], Loss: 0.483
[Epoch, Batch]: [4, 200], Loss: 0.474
[Epoch, Batch]: [4, 300], Loss: 0.478
Validation Loss: 0.595, Accuracy: 81.88%
[Epoch, Batch]: [5, 100], Loss: 0.393
[Epoch, Batch]: [5, 200], Loss: 0.401
[Epoch, Batch]: [5, 300], Loss: 0.387
Validation Loss: 0.568, Accuracy: 82.38%
[Epoch, Batch]: [6, 100], Loss: 0.316
[Epoch, Batch]: [6, 200], Loss: 0.331
[Epoch, Batch]: [6, 300], Loss: 0.338
Validation Loss: 0.528, Accuracy: 84.08%
[Epoch, Batch]: [7, 100], Loss: 0.281
[Epoch, Batch]: [7, 200], Loss: 0.270
[Epoch, Batch]: [7, 300], Loss: 0.291
Validation Loss: 0.552, Accuracy: 83.08%
[Epoch, Batch]: [8, 100], Los

In [8]:
for epoch in range(10):  # Adjust the number of epochs as needed
    train(model, trainloader, criterion, optimizer, device)
    val_loss, _ = validate(model, valloader, criterion, device)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

[Epoch, Batch]: [1, 100], Loss: 0.143
[Epoch, Batch]: [1, 200], Loss: 0.156
[Epoch, Batch]: [1, 300], Loss: 0.160
Validation Loss: 0.549, Accuracy: 84.38%
Early stopping triggered.


In [9]:
def evaluate():
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('Accuracy of the network on the 10000 test images: %d %%' % accuracy)

evaluate()

Accuracy of the network on the 10000 test images: 84 %
